# Imports

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import hpc05
import holoviews as hv

import numpy as np
import kwant
import scipy.constants
import cmath
import functools as ft
import adaptive
adaptive.notebook_extension()

import sns_system, spectrum, topology, supercurrent

from IPython.core.magic import register_cell_magic
import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)

interactive_shell = get_ipython()
@register_cell_magic
def cluster_mark_checked(line, cell):
    if run_cluster.value is True:
        run_cluster.value = False
        interactive_shell.run_cell(cell)
    else:
        return "Command not run. Check the box above to run."

del(cluster_mark_checked)
run_cluster

In [ ]:
import numpy as np
import scipy.constants
import cmath

import functools as ft
import kwant
import sns_system, spectrum, topology, supercurrent


In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Make zigzag shaped system

In [ ]:
def zigzag_shaped_system(a, L_m, L_x, z_x, z_y, W_up, W_down, edge_thickness=1,
                    transverse_soi=True,
                    mu_from_bottom_of_spin_orbit_bands=True,
                    k_x_in_sc=False, leaded=True, **_):

    #     HAMILTONIAN DEFINITIONS
    template_strings = sns_system.get_template_strings(
        transverse_soi, mu_from_bottom_of_spin_orbit_bands, k_x_in_sc)
    
    conservation_matrix = -supercurrent.sigz
    
    # TURN HAMILTONIAN STRINGS INTO TEMPLATES
    kwargs = dict(coords=('x', 'y'), grid_spacing=a)
    template_barrier = kwant.continuum.discretize(
        template_strings['ham_barrier'], **kwargs)
    template_normal = kwant.continuum.discretize(
        template_strings['ham_normal'], **kwargs)
    template_sc_left = kwant.continuum.discretize(
        template_strings['ham_sc_left'], **kwargs)
    template_sc_right = kwant.continuum.discretize(
        template_strings['ham_sc_right'], **kwargs)

    def union_shape(shapes):
        def _shape(pos):
            res = False
            for shape in shapes:
                res |= shape(pos)
            return res
        return _shape
        
    def intersection_shape(shape_A, shape_B):
        def _shape(pos):
            return shape_A(pos) and not shape_B(pos)
        return _shape
    
    def below_zigzag(z_x, z_y, offset):
        def shape(site):
            x, y = site.pos - offset
            if 0 <= x < 2*z_x:
                return y <  a * ((z_y* np.sin(np.pi*2 * x / z_x))//a)
            else:
                return False
        return shape
    
    def above_zigzag(z_x, z_y, offset):
        def shape(pos):
            return not below_zigzag(z_x, z_y, offset)(pos)
        return shape
    
    def within_zigzag(z_x, z_y, offset):
        def shape(pos):
            x, y = pos.pos
            return below_zigzag(z_x, z_y, (offset[0], offset[1] + L_m))(pos) and above_zigzag(z_x, z_y, offset)(pos) and 0<=x<L_x
        return shape
    
    def edge_zigzag(z_x, z_y, offset):
        def shape(pos):
            x, y = pos.pos
            return ((below_zigzag(z_x, z_y, (offset[0], offset[1] + L_m))(pos) and above_zigzag(z_x, z_y, (offset[0], offset[1] + L_m-a*edge_thickness))(pos) and 0<=x<L_x)
                    or
                   (below_zigzag(z_x, z_y, (offset[0], offset[1] +edge_thickness*a))(pos) and above_zigzag(z_x, z_y, (offset[0], offset[1]))(pos) and 0<=x<L_x))
        return shape
    
    number_of_zigzags = int(L_x // (2*z_x))+1
    within_zigzag_shapes = [within_zigzag(z_x, z_y, (2*z_x*i, 0)) for i in range(number_of_zigzags)]
    edge_zigzag_shapes = [edge_zigzag(z_x, z_y, (2*z_x*i, 0)) for i in range(number_of_zigzags)]
    edge_shape =union_shape(edge_zigzag_shapes)
    
    middle_shape_with_edge = union_shape(within_zigzag_shapes)
    middle_shape = intersection_shape(middle_shape_with_edge, edge_shape)
    
    def top_shape_block(site):
        x, y = site.pos
        return 0 <= x < L_x and -z_y <= y < L_m + z_y + W_up
    top_shape_with_some_down = intersection_shape(top_shape_block, middle_shape_with_edge)
    
    def down_shape_block(site):
        x, y = site.pos
        return 0 <= x < L_x and -W_down - z_y <= y < z_y
    down_shape_with_some_top = intersection_shape(down_shape_block, middle_shape_with_edge)
    
    top_shape = top_shape_with_some_down
    down_shape = down_shape_with_some_top
    
    # BUILD FINITE SYSTEM
    syst = kwant.Builder()
    syst.fill(template_normal, middle_shape, (0, L_m//2))
    
    if edge_thickness ==0:
        pass
    
    elif edge_thickness == 1:
        for x in np.arange(0, L_x, a):
            y_up = ((z_y* np.sin(np.pi*2 * x / z_x)+L_m)//a - 1)*a
            y_down = ((z_y* np.sin(np.pi*2 * x / z_x))//a)*a
#             if (x//z_x)%2 == 1:
#                 y = z_y - y
            syst.fill(template_barrier, edge_shape, (x, y_up))
            syst.fill(template_barrier, edge_shape, (x, y_down))
    
    else:  
        syst.fill(template_barrier, edge_shape, (0, 0))
        syst.fill(template_barrier, edge_shape, (0, L_m-a))
    
    if W_up is not 0:
        syst.fill(template_sc_left, top_shape, (0, L_m))
    syst.fill(template_sc_right, down_shape, (0, -a))
    
    syst = syst.finalized()

    return syst

In [ ]:
def zigzag_system_lead(a, L_m, L_x, z_x, z_y, W_up, W_down, edge_thickness=1,
                    transverse_soi=True,
                    mu_from_bottom_of_spin_orbit_bands=True,
                    k_x_in_sc=False, leaded=True, **_):

    #     HAMILTONIAN DEFINITIONS
    template_strings = sns_system.get_template_strings(
        transverse_soi, mu_from_bottom_of_spin_orbit_bands, k_x_in_sc)
    
    conservation_matrix = -supercurrent.sigz
    
    # TURN HAMILTONIAN STRINGS INTO TEMPLATES
    kwargs = dict(coords=('x', 'y'), grid_spacing=a)
    template_barrier = kwant.continuum.discretize(
        template_strings['ham_barrier'], **kwargs)
    template_normal = kwant.continuum.discretize(
        template_strings['ham_normal'], **kwargs)
    template_sc_left = kwant.continuum.discretize(
        template_strings['ham_sc_left'], **kwargs)
    template_sc_right = kwant.continuum.discretize(
        template_strings['ham_sc_right'], **kwargs)

    def union_shape(shapes):
        def _shape(pos):
            res = False
            for shape in shapes:
                res |= shape(pos)
            return res
        return _shape
        
    def intersection_shape(shape_A, shape_B):
        def _shape(pos):
            return shape_A(pos) and not shape_B(pos)
        return _shape
    
    def below_zigzag(z_x, z_y, offset):
        def shape(site):
            x, y = site.pos - offset
            if 0 <= x < 2*z_x:
                return y <  a * ((z_y* np.sin(np.pi*2 * x / z_x))//a)
            else:
                return False
        return shape
    
    def above_zigzag(z_x, z_y, offset):
        def shape(pos):
            return not below_zigzag(z_x, z_y, offset)(pos)
        return shape
    
    def within_zigzag(z_x, z_y, offset):
        def shape(pos):
            x, y = pos.pos
            return below_zigzag(z_x, z_y, (offset[0], offset[1] + L_m))(pos) and above_zigzag(z_x, z_y, offset)(pos) and 0<=x<L_x
        return shape
    
    def edge_zigzag(z_x, z_y, offset):
        def shape(pos):
            x, y = pos.pos
            return ((below_zigzag(z_x, z_y, (offset[0], offset[1] + L_m))(pos) and above_zigzag(z_x, z_y, (offset[0], offset[1] + L_m-a*edge_thickness))(pos) and 0<=x<L_x)
                    or
                   (below_zigzag(z_x, z_y, (offset[0], offset[1] +edge_thickness*a))(pos) and above_zigzag(z_x, z_y, (offset[0], offset[1]))(pos) and 0<=x<L_x))
        return shape
    
    number_of_zigzags = int(L_x // (2*z_x))+1
    within_zigzag_shapes = [within_zigzag(z_x, z_y, (2*z_x*i, 0)) for i in range(number_of_zigzags)]
    edge_zigzag_shapes = [edge_zigzag(z_x, z_y, (2*z_x*i, 0)) for i in range(number_of_zigzags)]
    edge_shape =union_shape(edge_zigzag_shapes)
    
    middle_shape_with_edge = union_shape(within_zigzag_shapes)
    middle_shape = intersection_shape(middle_shape_with_edge, edge_shape)
    
    def top_shape_block(site):
        x, y = site.pos
        return 0 <= x < L_x and -z_y <= y < L_m + z_y + W_up
    top_shape_with_some_down = intersection_shape(top_shape_block, middle_shape_with_edge)
    
    def down_shape_block(site):
        x, y = site.pos
        return 0 <= x < L_x and -W_down - z_y <= y < z_y
    down_shape_with_some_top = intersection_shape(down_shape_block, middle_shape_with_edge)
    
    top_shape = top_shape_with_some_down
    down_shape = down_shape_with_some_top
    
    def shape_dummy(site):
        x, y = site.pos
        return x==0 and -W_down - z_y <= y < L_m
    
    # BUILD FINITE SYSTEM
    syst = kwant.Builder(kwant.TranslationalSymmetry([L_x, 0]))
    dummy = kwant.Builder()
    dummy.fill(template_normal, shape_dummy, (0, L_m//2))
    
    syst.fill(template_normal, middle_shape, (0, L_m//2))
    
    if edge_thickness ==0:
        pass
    
    elif edge_thickness == 1:
        for x in np.arange(0, L_x, a):
            y_up = ((z_y* np.sin(np.pi*2 * x / z_x)+L_m)//a - 1)*a
            y_down = ((z_y* np.sin(np.pi*2 * x / z_x))//a)*a

            syst.fill(template_barrier, edge_shape, (x, y_up))
            syst.fill(template_barrier, edge_shape, (x, y_down))
    
    else:  
        syst.fill(template_barrier, edge_shape, (0, 0))
        syst.fill(template_barrier, edge_shape, (0, L_m-a))
    
    if W_up is not 0:
        syst.fill(template_sc_left, top_shape, (0, L_m))
    syst.fill(template_sc_right, down_shape, (0, -a))

    return kwant.wraparound.wraparound(syst).finalized()

# Bandstructure normal system

In [ ]:
import plotting_results

In [ ]:
params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = 0*1.0341,
                 phase = 0*np.pi,
                 T = 0.0,
                 V = 0)

syst_pars = dict(a=10, L_m=200, L_x=10, W_up=200, W_down=200, z_x=50, z_y=0,
                 offset_lead=100, k_x_in_sc=True, leaded=False, edge_thickness=0)

params = dict(**constants,
              **params_raw)

from formulas import *
kf = fermi_wavenumber(params['mu'], 0.02)*syst_pars['a']

syst=zigzag_system_lead(**syst_pars)
plotting_results.plot_syst(syst=syst, params=dict(k_x=0,**params));

In [ ]:
def bands_wrap(k):
    ham = syst.hamiltonian_submatrix(sparse=True, params=dict(**params, k_x=k))
    e, ev = spectrum.sparse_diag(ham, 100, 0)
    return np.sort(e)
    
learner = adaptive.Learner1D(bands_wrap, [-np.pi, np.pi])
runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
kf

In [ ]:
learner.plot()[-1:1, -4:4].redim(x='k', y='E')*hv.Path(([-kf, -kf],[-4,4]))*hv.Path(([kf, kf],[-4,4]))

# Why does minimum of energy occur around $k_x = k_F$

To keep the kinetic term in the hamiltonian as low as possible, one generally should minimize the magnitude of $\vec{k}$. 
The lowest possible accesible mode is: $\rvert\vec{k}\rvert = k_F$. Of course this leaves us to choose  the angle of $\vec{k}$. Which angle has the lowest energy?

In [ ]:
syst_pars = dict(a=20, L_m=200, L_x=1000, W_up=200, W_down=200, z_x=50, z_y=0,
                 offset_lead=100, k_x_in_sc=True, leaded=False, edge_thickness=0)

params = dict(**constants,
              **params_raw)

syst=zigzag_system_lead(**syst_pars)
plotting_results.plot_syst(syst=syst, params=dict(k_x=0,**params));

# Snake like system

In [ ]:
syst_pars = dict(a=10, L_m=200, L_x=1000, W_up=200, W_down=200, z_x=1000, z_y=125,
                 offset_lead=100, k_x_in_sc=True, leaded=False, edge_thickness=0)

params = dict(**constants,
              **params_raw)

from formulas import *
kf = fermi_wavenumber(params['mu'], 0.02)*syst_pars['a']

syst=zigzag_system_lead(**syst_pars)
plotting_results.plot_syst(syst=syst, params=dict(k_x=0,**params));

In [ ]:
def bands_wrap(k):
    ham = syst.hamiltonian_submatrix(sparse=True, params=dict(**params, k_x=k))
    e, ev = spectrum.sparse_diag(ham, 100, 0)
    return np.sort(e)
    
learner = adaptive.Learner1D(bands_wrap, [-np.pi*syst_pars['a']/syst_pars['L_x'], np.pi*syst_pars['a']/syst_pars['L_x']])
runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
runner.cancel()

In [ ]:
learner.plot()

In [ ]:
ham = syst.hamiltonian_submatrix(params=dict(**params, k_x=0), sparse=True)
d = kwant.operator.Density(syst)
e, ev = spectrum.sparse_diag(ham,4,0)

In [ ]:
e

In [ ]:
ax=plt.axes()
plotting_results.plot_syst(syst=syst, params=dict(**params, k_x=0),ax=ax)
kwant.plotter.map(syst, d(ev[:,0]), ax=ax)
# plt.savefig('no_ohase_difference', dpi=1000)